In [1]:
import pandas as pd
import numpy as np
import json
import sqlite3
from quizzes import process_quiz

db_file = 'progsnap2_21_consenting.db'
NUM_QUIZ_INCLUDE = 10

# Subject table
df_LinkSubject = pd.read_sql(f"""
SELECT * FROM LinkSubject
""", sqlite3.connect(db_file))

# Assignment table
df_LinkAssignment = pd.read_sql(f"""
SELECT * FROM LinkAssignment
""", sqlite3.connect(db_file))

In [2]:
# Split students' final grade into high and low based on the median
df_high_low = df_LinkSubject[['SubjectID', 'X-TrueLetterGrade']]
df_high_low['HighLow'] = (df_high_low['X-TrueLetterGrade'].isin(['A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-'])).astype(int)
df_high_low

C:\Users\mianh\AppData\Local\Temp/ipykernel_2760/21950002.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_high_low['HighLow'] = (df_high_low['X-TrueLetterGrade'].isin(['A', 'A-', 'B+', 'B', 'B-', 'C+', 'C', 'C-'])).astype(int)


,SubjectID,X-TrueLetterGrade,HighLow
0,630,A-,1
1,638,A,1
2,571,C,1
3,575,A-,1
4,617,C-,1
...,...,...,...
99,637,A,1
100,633,C+,1
101,815,F,0
102,737,F,0


In [3]:
# Get subject ID, 295
list_subjectID = df_LinkSubject.loc[:,'SubjectID']

# Get quiz ID, 384 388 434
list_quizID = df_LinkAssignment.loc[df_LinkAssignment['X-Name'].str.contains('Quiz'), 'AssignmentId']
list_quizID = list_quizID[:NUM_QUIZ_INCLUDE]

In [4]:
# Filter first 10 quizzes, with all subject ID
subject_query = ("'" + list_subjectID + "'").to_string(index=False).replace('\n',',')
quiz_query = ("'" + list_quizID + "'").to_string(index=False).replace('\n',',')

df_quiz = pd.read_sql(f"""
SELECT MainTable.SubjectID, MainTable.AssignmentID, `X-Instructions` as Instructions, `X-Code.OnRun` as OnRun, Contents, EventType
FROM MainTable, CodeState, LinkAssignment
WHERE CodeState.ID = MainTable.CodeStateID
AND MainTable.CodeStateID != '0'
AND MainTable.AssignmentID in ({quiz_query})
AND MainTable.SubjectID in ({subject_query})
AND LinkAssignment.AssignmentID = MainTable.AssignmentID
""", sqlite3.connect(db_file))
df_quiz

,SubjectID,AssignmentID,Instructions,OnRun,Contents,EventType
0,577,384,"{""questions"":{""Define Computer Science"":{""type...","{""questions"":{""Define Computer Science"":{""corr...","b'{\n ""studentAnswers"": {\n ""Define Comput...",File.Edit
1,577,384,"{""questions"":{""Define Computer Science"":{""type...","{""questions"":{""Define Computer Science"":{""corr...","b'{\n ""studentAnswers"": {\n ""Define Comput...",File.Edit
2,577,384,"{""questions"":{""Define Computer Science"":{""type...","{""questions"":{""Define Computer Science"":{""corr...","b'{\n ""studentAnswers"": {\n ""Define Comput...",File.Edit
3,577,384,"{""questions"":{""Define Computer Science"":{""type...","{""questions"":{""Define Computer Science"":{""corr...","b'{\n ""studentAnswers"": {\n ""Define Comput...",File.Edit
4,577,384,"{""questions"":{""Define Computer Science"":{""type...","{""questions"":{""Define Computer Science"":{""corr...","b'{\n ""studentAnswers"": {\n ""Define Comput...",File.Edit
...,...,...,...,...,...,...
32816,808,345,"{\n ""questions"": {\n ""ArithmeticOperatorPr...","{\n ""questions"": {\n ""ArithmeticOperatorPr...","b'{\n ""studentAnswers"": {\n ""ArithmeticOpe...",X-Submission.LMS
32817,808,345,"{\n ""questions"": {\n ""ArithmeticOperatorPr...","{\n ""questions"": {\n ""ArithmeticOperatorPr...","b'{\n ""studentAnswers"": {\n ""ArithmeticOpe...",File.Edit
32818,808,345,"{\n ""questions"": {\n ""ArithmeticOperatorPr...","{\n ""questions"": {\n ""ArithmeticOperatorPr...","b'{\n ""studentAnswers"": {\n ""ArithmeticOpe...",File.Edit
32819,808,345,"{\n ""questions"": {\n ""ArithmeticOperatorPr...","{\n ""questions"": {\n ""ArithmeticOperatorPr...","b'{\n ""studentAnswers"": {\n ""ArithmeticOpe...",File.Edit


In [14]:
for index in range(9):
    df_single_quiz = df_quiz.loc[df_quiz['AssignmentID'].str.contains(list_quizID.iloc[index])].reset_index()
    results = df_single_quiz.apply(lambda row: 
        vars(process_quiz(json.loads(row.Instructions),
                          json.loads(row.OnRun),
                          json.loads(row.Contents.decode('utf-8') or "{}"))),
        axis=1
    )
    aaa = pd.json_normalize(results)
    matching = [s for s in list(aaa.columns) if ".score" in s]
    bbb = aaa.loc[:,matching]
    bbb = bbb.astype(float)
    bbb = bbb.fillna(0).astype(float)

    df_single_quiz_state = pd.concat([df_single_quiz[['SubjectID']], bbb], axis=1)
    df_single_quiz_state
    f_high = open("quiz"+str(index+1)+"_high.txt", "a")
    f_low = open("quiz"+str(index+1)+"_low.txt", "a")
    f_id = open("quiz"+str(index+1)+"_id.txt", "a")
    m=1
    n=1
    for j in range(len(list_subjectID)):
        ccc = df_single_quiz_state.loc[
            df_single_quiz_state['SubjectID'].str.contains(list_subjectID[j]), 
            matching].to_numpy()
        np.set_printoptions(formatter={'float': '{: 0.1f}'.format})
        if(df_high_low.loc[j, 'HighLow'] == 1):
            print('subject id is high', list_subjectID[j])
            f_id.write('high '+ list_subjectID[j] + '\n')
            f_high.write('s'+str(m)+' '+np.array2string(ccc).replace('  ', '_').replace(']\n [', '')[3:-2] + '\n')
            m=m+1
        else:
            print('subject id is low', list_subjectID[j])
            f_id.write('low '+ list_subjectID[j] + '\n')
            f_low.write('s'+str(n)+' '+np.array2string(ccc).replace('  ', '_').replace(']\n [', '')[3:-2] + '\n')
            n=n+1
    f_high.close()
    f_low.close()
    f_id.close()

subject id is high 630
subject id is high 638
subject id is high 571
subject id is high 575
subject id is high 617
subject id is low 743
subject id is high 605
subject id is high 603
subject id is low 598
subject id is high 648
subject id is high 723
subject id is high 585
subject id is high 806
subject id is high 729
subject id is low 616
subject id is high 663
subject id is high 624
subject id is high 674
subject id is low 726
subject id is high 636
subject id is low 449
subject id is low 812
subject id is low 620
subject id is high 584
subject id is high 627
subject id is high 754
subject id is low 664
subject id is high 577
subject id is high 581
subject id is low 804
subject id is high 626
subject id is high 785
subject id is high 696
subject id is high 625
subject id is low 685
subject id is high 719
subject id is low 258
subject id is high 1036
subject id is low 814
subject id is high 650
subject id is high 676
subject id is high 295
subject id is high 837
subject id is high 690

subject id is high 630
subject id is high 638
subject id is high 571
subject id is high 575
subject id is high 617
subject id is low 743
subject id is high 605
subject id is high 603
subject id is low 598
subject id is high 648
subject id is high 723
subject id is high 585
subject id is high 806
subject id is high 729
subject id is low 616
subject id is high 663
subject id is high 624
subject id is high 674
subject id is low 726
subject id is high 636
subject id is low 449
subject id is low 812
subject id is low 620
subject id is high 584
subject id is high 627
subject id is high 754
subject id is low 664
subject id is high 577
subject id is high 581
subject id is low 804
subject id is high 626
subject id is high 785
subject id is high 696
subject id is high 625
subject id is low 685
subject id is high 719
subject id is low 258
subject id is high 1036
subject id is low 814
subject id is high 650
subject id is high 676
subject id is high 295
subject id is high 837
subject id is high 690

subject id is low 659
subject id is high 632
subject id is high 673
subject id is low 746
subject id is high 680
subject id is high 675
subject id is high 646
subject id is high 587
subject id is high 610
subject id is high 808
subject id is high 749
subject id is high 686
subject id is low 700
subject id is high 677
subject id is high 671
subject id is high 535
subject id is high 590
subject id is low 525
subject id is high 1243
subject id is high 735
subject id is high 684
subject id is high 728
subject id is low 807
subject id is low 435
subject id is high 637
subject id is high 633
subject id is low 815
subject id is low 737
subject id is low 509
subject id is high 630
subject id is high 638
subject id is high 571
subject id is high 575
subject id is high 617
subject id is low 743
subject id is high 605
subject id is high 603
subject id is low 598
subject id is high 648
subject id is high 723
subject id is high 585
subject id is high 806
subject id is high 729
subject id is low 616